# 自动微分

在梯度计算中，梯度是损失函数关于模型参数（权重、偏差）的偏导数。

通常我们有两种直观的方法计算导数，但它们在深度学习面前都显得力不从心。

1. 数值微分：

我们可以通过数值模拟的方法，给每个参数增加一个微小的扰动$ϵ$来近似计算梯度：

$$
\frac{\partial J}{\partial w_i} \approx \frac{J(w_i + \epsilon) - J(w_i)}{\epsilon}
$$

但这种方法在深度学习中是不可接受的。假设我们的网络有100万个参数，为了得到全部梯度，我们需要进行100万次前向传播计算（即运行100万次模型推理），这会造成灾难性的计算开销。

2. 符号微分：

另一种方法，是像我们在第一部分关于多层神经元网络的章节里做的那样，手动推导出损失函数的数学解析式。

但是对于简单的线性回归，手动推导很容易。如果我们有100层网络，包含各种更复杂的变换，手动推导出的导数公式将长达数页。而一旦修改了网络结构（比如加个层），所有公式都要推倒重来，极其缺乏灵活性。

在深度学习的实践中，普遍采用的是第三种方法：

3. 自动微分：

**自动微分**（Automatic Differentiation）的实现依据是微积分的**链式法则**（Chain Rule）；它的设计思路是：任何复杂的运算，在计算机底层都是由一系列基本运算（如加、减、乘、除、指数、对数）组合而成的。

* 分解计算图：我们将复杂的公式拆解为一个计算图（Computational Graph）。每一个节点只负责一个极简单的基本运算。
* 局部求导：对于每个基本运算（如$y=a⋅b$），我们预先定义好它的求导规则。
* 根据微积分的链式法则：

$$
\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}
$$

当我们从损失函数开始梯度计算时，模型并不需要知道所有导数的完整复杂公式，它只需要从损失值开始倒着走：
* 拿到当前层对下一层的梯度（上游梯度）。
* 乘上当前算子的局部导数（Local Gradient）。
* 结果传回给上一层节点。

通过这种方式，我们只需要执行一次前向传播（记录路径）和一次梯度计算（应用链式法则），就能以极高的精度计算出所有参数的梯度。


In [44]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

张量不仅是保存数据的地方，也是进行梯度计算，和计算链传播的地方。为此，我们增加了三个变量：

* grad：保存梯度
* gradient_fn：局部导数计算函数
* parents：上一层节点（构成梯度计算链）

同时，我们添加了一个函数：
* backward()：调用gradient_fn计算局部梯度，并调用parents中所有上一层节点进行梯度计算。这是一个递归调用过程，将触发所有上级节点，完成全部梯度计算。

In [45]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### 基础层

In [46]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def __str__(self):
        return ''

### 基础损失函数

In [47]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

## 数据

### 特征、标签

In [48]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

在线性层，我们要在**预测值张量**中添加线性回归的梯度计算函数，并将结果分别保存在**权重**和**偏差**的梯度变量（grad）中。

In [49]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        return p

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### 损失函数（平均平方差）

损失函数是梯度计算的起点。所以我们也需要在**损失值张量**中添加平均平方差的梯度计算函数，并将结果保存在**预测值**的梯度变量（grad）中。

同时，要将预测值添加到上一级节点的列表中。

In [50]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

## 验证

### 推理

In [51]:
layer = Linear(feature.size(), 1)
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [52]:
loss = MSELoss()
error = loss(prediction, label)
print(f'error: {error}')

error: 14871.802500000002


## 训练

### 梯度计算

现在，我们在梯度计算的时候，只需要调用损失值张量中的梯度计算函数。它将完成预测值梯度的计算，并传递到预测值张量完成权重和偏差的梯度计算。

In [53]:
error.backward()

### 反向传播

在反向传播的步骤，我们需要让每个张量减去它的的梯度值，就完成了一次迭代。

In [54]:
layer.weight.data -= layer.weight.grad
layer.bias.data -= layer.bias.grad
print(layer)

weight: [[ 6854.09 14146.7 ]]
bias: [243.9]


### 重新评估

In [55]:
prediction = layer(feature)
error = loss(prediction, label)
print(f'error: {error}')

error: 1026548766283.6302
